In [ ]:
! pip install scikit-learn

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
]
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
X

# Part 3: Stemming


In [ ]:
!pip install spacy

In [133]:
#importing libraries
import spacy
from typing import List


texts = [
    "Net income was $9.4 million compared to the prior year of $2.7 million.",
    "Revenue exceeded twelve billion dollars, with a loss of $1b.",
]

nlp = spacy.load("en_core_web_sm")
#for doc in nlp.pipe(texts, disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"]):
    # Do something with the doc here
 #    print([(ent.text, ent.label_) for ent in doc.ents])


pipeline = ["lemmatizer"]
disable = ['tok2vec', 'tagger', 'parser', 'senter', 'attribute_ruler', 'ner']

print(nlp.pipe_names)
for doc in nlp.pipe(texts, disable=disable ,n_process=4):
    print(doc.ents["parser"])

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']


/Users/quentinfisch/Documents/EPITA/ING2/SCIA/S8/NLP1/nlp1-lab2/.venv/lib/python3.9/site-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


TypeError: tuple indices must be integers or slices, not str

In [52]:
! python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 7.7 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


NLP LAB 02
Théo Ripoll - Quentin Fish - Nicolas

Loading the dataset

## Part 1: The Dataset

In [ ]:
from datasets import load_dataset
from datasets import get_dataset_split_names
import pandas as pd

In [ ]:
dataset = load_dataset("imdb")
dataset

In [ ]:
get_dataset_split_names("imdb")

In [ ]:
#Count the number of different labels in each datasets
train_labels = pd.DataFrame(dataset["train"]['label'], columns=["label"])
print(train_labels.groupby("label")["label"].count())

test_labels = pd.DataFrame(dataset["test"]['label'], columns=["label"])
print(test_labels.groupby("label")["label"].count())

### Question 1: How many splits does the dataset has?
There are 3 splits: train, test and unsupervised

### Question 2: How big are the splits ?
train: 25000
test: 25000
unsupervised: 50000

### Question 3: What is the proportion of each class on the supervised splits?
train: 50% positive, 50% negative
test: 50% positive, 50% negative

## Partie 2: Naive Bayes classifier

In [127]:
from string import punctuation
import re
# Prepare the dataset

def preprocess(dataset: pd.DataFrame) -> pd.DataFrame : 
    # First lower the case
    dataset["text"] = dataset["text"].apply(lambda x: x.lower())
    # Replace the punctuation with spaces. We keep the . ? ! that may give revelant informations
    punctuation_to_remove = "|".join(list(filter(lambda p: p != '.' and p != '!' and p != '?',punctuation)))
    print(punctuation_to_remove)
    dataset["text"] = dataset["text"].apply(lambda x: re.sub(punctuation_to_remove, " ", x))
    return dataset

preprocessed = preprocess(pd.DataFrame(dataset["train"], columns=["text", "label"]).loc[:1, :],)
preprocessed

"|#|$|%|&|'|(|)|*|+|,|-|/|:|;|<|=|>|@|[|\|]|^|_|`|{|||}|~


error: nothing to repeat at position 16

In [98]:
from string import punctuation


'"|#|$|%|&|\'|(|)|*|+|,|-|/|:|;|<|=|>|@|[|\\|]|^|_|`|{|||}|~'

### Classifier using pseudo-code

In [84]:
train_dataset_docs = pd.DataFrame(dataset["train"]["text"], columns=["document"])
train_dataset_classes = pd.DataFrame(dataset["train"]["label"], columns=["class"])
train_dataset = pd.concat([train_dataset_docs, train_dataset_classes], axis=1)
train_dataset

,document,class
0,I rented I AM CURIOUS-YELLOW from my video sto...,0
1,"""I Am Curious: Yellow"" is a risible and preten...",0
2,If only to avoid making this type of film in t...,0
3,This film was probably inspired by Godard's Ma...,0
4,"Oh, brother...after hearing about this ridicul...",0
...,...,...
24995,A hit at the time but now better categorised a...,1
24996,I love this movie like no other. Another time ...,1
24997,This film and it's sequel Barry Mckenzie holds...,1
24998,'The Adventures Of Barry McKenzie' started lif...,1


In [130]:
import numpy as np

def get_vocabulary(dataset: pd.DataFrame, min_freq: int = 0) -> List[str]:
    vectorizer = CountVectorizer(min_df=min_freq)
    vectorizer.fit(dataset["document"])
    return vectorizer.vocabulary_

def train_naive_bayes(dataset):
    classes = dataset["class"].unique()
    logprior = {}
    bigdoc = {}
    count = {}
    loglikelihood = {}
    V = get_vocabulary(dataset)
    for c in classes:
        n_doc = len(dataset)
        n_c = len(dataset[dataset["class"] == c])
        logprior[c] = np.log(n_c / n_doc)
        bigdoc[c] = " ".join(dataset[dataset["class"] == c]["document"])
        for word in V:
            count[(word, c)] = bigdoc[c].count(word)
            loglikelihood[(word, c)] = np.log((count[(word, c)] + 1) / (sum(count.values()) + len(V)))
    return logprior, loglikelihood, V

def test_naive_bayes(dataset, logprior, loglikelihood, V):
    classes = dataset["class"].unique()
    sum_loglikelihood = {}
    for c in classes:
        sum_loglikelihood[c] = logprior[c]
        for word in V:
            if word in dataset["document"]:
                sum_loglikelihood[c] += loglikelihood[(word, c)]
    return max(sum_loglikelihood, key=sum_loglikelihood.get)


In [131]:
train_naive_bayes(train_dataset)

KeyboardInterrupt: 